In [1]:
#import the libraries
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
#load all the datasets we have created previously

load_data_inputs_train= pd.read_csv('/content/drive/MyDrive/credit_risk_analysis/Data/loan_data_inputs_train.csv', index_col= 0)
load_data_targets_train= pd.read_csv('/content/drive/MyDrive/credit_risk_analysis/Data/loan_data_targets_train.csv',index_col= 0)
load_data_inputs_test= pd.read_csv('/content/drive/MyDrive/credit_risk_analysis/Data/loan_data_inputs_test.csv', index_col= 0)
load_data_targets_test= pd.read_csv('/content/drive/MyDrive/credit_risk_analysis/Data/loan_data_targets_test.csv', index_col= 0)

In [4]:
#view the inputs dataframe
display(load_data_inputs_train.head())
display(load_data_inputs_test.head())

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,dti:25-30,dti:30-35,dti:35-40,dti:>40,mths_since_last_record_factor,mths_since_last_record:Missing,mths_since_last_record:0-3,mths_since_last_record:3-85,mths_since_last_record:85-120,mths_since_last_record:>=120
403678,14560583,16622883,24000,24000,24000.0,60 months,11.99,533.75,B,B3,...,0,0,0,0,NaN,1,0,0,0,0
241492,34303344,36966688,30000,30000,30000.0,60 months,12.39,673.27,C,C1,...,0,0,0,0,NaN,1,0,0,0,0
15864,737124,934233,2400,2400,2400.0,36 months,13.49,81.44,C,C2,...,0,0,0,0,NaN,1,0,0,0,0
320277,25326009,27668971,12000,12000,12000.0,36 months,13.98,410.02,C,C3,...,0,0,0,0,NaN,1,0,0,0,0
299829,28633305,31166460,5000,5000,5000.0,36 months,11.67,165.29,B,B4,...,0,0,0,0,"(79.98, 82.56]",0,0,1,0,0


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,dti:25-30,dti:30-35,dti:35-40,dti:>40,mths_since_last_record_factor,mths_since_last_record:Missing,mths_since_last_record:0-3,mths_since_last_record:3-85,mths_since_last_record:85-120,mths_since_last_record:>=120
421662,13105863,15128065,16000,16000,16000.0,36 months,12.49,535.19,B,B4,...,0,0,0,0,"(10.32, 12.9]",0,0,1,0,0
345847,21280193,23583077,24000,24000,24000.0,60 months,16.29,587.34,D,D2,...,0,0,0,0,NaN,1,0,0,0,0
87245,7436827,9098912,19125,19125,19125.0,60 months,23.10,540.25,E,E4,...,1,0,0,0,NaN,1,0,0,0,0
422411,12968478,15000658,15000,15000,15000.0,36 months,13.65,510.12,C,C1,...,0,0,0,0,"(74.82, 77.4]",0,0,1,0,0
131614,5616662,6998902,24000,24000,24000.0,60 months,23.76,687.10,F,F4,...,0,0,0,0,NaN,1,0,0,0,0


In [5]:
#view the target datasets
display(load_data_targets_test.head())
display(load_data_targets_train.head())

,good_bad
421662,1
345847,1
87245,1
422411,1
131614,1


,good_bad
403678,1
241492,1
15864,0
320277,1
299829,1


In [6]:
#view the shape of train & test inputs dataframe
display(load_data_inputs_train.shape)
display(load_data_inputs_test.shape)

(373028, 352)

(93257, 352)

In [7]:
#view the shape of train & test targets dataframe
display(load_data_targets_train.shape)
display(load_data_targets_test.shape)

(373028, 1)

(93257, 1)

In [8]:
#pd.options.display.max_rows = None
# get the list of columns to select our features in next step
display(list(load_data_inputs_train.columns))

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [9]:
# list of features created before in preprocessing steps(removeing factor columns)
feature_selection= [
 'loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'grade:A',
 'grade:B',
 'grade:C',
 'grade:D',
 'grade:E',
 'grade:F',
 'grade:G',
 'home_ownership:RENT_OTHER_NONE_ANY',
 'home_ownership:MORTGAGE',
 'home_ownership:OWN',
 'verification_status:Not Verified',
 'verification_status:Source Verified',
 'verification_status:Verified',
 'initial_list_status:f',
 'initial_list_status:w',
 'addr_state_NE',
 'addr_state_ME',
 'addr_state_IA',
 'addr_state_NV',
 'addr_state_HI_FL_AL',
 'addr_state_NY_LA_NM_OK_NC_MO_MD_NJ_VA_CA_AZ_MI_UT_TN_AR_PA_RI_OH_KY_DE_MN_SD_MA_IN_GA',
 'addr_state_WA_WI_OR_TX_IL_CT_MT_CO_SC',
 'addr_state_KS_VT_AK_MS_ID_NH_WV',
 'addr_state_WY',
 'addr_state_DC',
 'purpose:educ_sm_b_ren_en_mov_house_other_med',
 'purpose:vac_wedd_debt_home_imp_major_pur_car_cc',
 'term:36',
 'term:60',
 'emp_length:0',
 'emp_length:1',
 'emp_length:2-4',
 'emp_length:5-6',
 'emp_length:7-9',
 'emp_length:10',
 'mths_since_issued_date:<116',
 'mths_since_issued_date:116-127',
 'mths_since_issued_date:127-138',
 'mths_since_issued_date:138-150',
 'mths_since_issued_date:150-161',
 'mths_since_issued_date:161-172',
 'mths_since_issued_date:172-183',
 'mths_since_issued_date:>183',
 'int_rate:<9.548',
 'int_rate:9.548-12.85',
 'int_rate:12.85-15.74',
 'int_rate:15.74-20.281',
 'int_rate:20.281',
 'mths_since_earliest_cr_line:<140',
 'mths_since_earliest_cr_line:140-239',
 'mths_since_earliest_cr_line:240-339',
 'mths_since_earliest_cr_line:340-439',
 'mths_since_earliest_cr_line:440-539',
 'mths_since_earliest_cr_line:540-600',
 'mths_since_earliest_cr_line:>600',
 #'installment<963.808',
 #'installment>=963.808',
 'delinq_2yrs:0',
 'delinq_2yrs:0-5',
 'delinq_2yrs:5-7',
 'delinq_2yrs:7-9',
 'delinq_2yrs:9-11',
 'delinq_2yrs:11-12',
 'delinq_2yrs:12-18',
 'delinq_2yrs:>18',
 'inq_last_6mths:0',
 'inq_last_6mths:0-4',
 'inq_last_6mths:4-6',
 'inq_last_6mths:6-9',
 'inq_last_6mths:9-11',
 'inq_last_6mths:11-13',
 'inq_last_6mths:>13',
 'open_acc:=0',
 'open_acc:0-4',
 'open_acc:4-26',
 'open_acc:26-28',
 'open_acc:28-33',
 'open_acc:33-39',
 'open_acc:>39',
 'pub_rec:0-6',
 'pub_rec:6-9',
 'pub_rec:9-10',
 'pub_rec>10',
 'total_acc:<=0.0-3.12',
 'total_acc:3.12-15.6',
 'total_acc:15.6-49.92',
 'total_acc:49.92-65.52',
 'total_acc:65.52-81.12',
 'total_acc:81.12-99.84',
 'total_acc:>99.84',
 'acc_now_delinq:0-1',
 'acc_now_delinq:1-2',
 'acc_now_delinq:2-3',
 'acc_now_delinq:>3',
 'total_rev_hi_lim:<6.4k',
 'total_rev_hi_lim:6.4k_12.8k',
 'total_rev_hi_lim:12.8k_20.8k',
 'total_rev_hi_lim:20.8k_30.4k',
 'total_rev_hi_lim:30.4k_35.2k',
 'total_rev_hi_lim:35.2k_46.4k',
 'total_rev_hi_lim:46.4k_56k',
 'total_rev_hi_lim:56k_70.4k',
 'total_rev_hi_lim:70.4k_78.4k',
 'total_rev_hi_lim:>78.4k',
 'annual_inc:<8k',
 'annual_inc:8k-30k',
 'annual_inc:30k-40k',
 'annual_inc:40k-50k',
 'annual_inc:50k-60k',
 'annual_inc:60k-70k',
 'annual_inc:70k-80k',
 'annual_inc:80k-90k',
 'annual_inc:90k-100k',
 'annual_inc:100k-120k',
 'annual_inc:120k-140k',
 'annual_inc:>140k',
 'mths_since_last_delinq:Missing',
 'mths_since_last_delinq:0-3',
 'mths_since_last_delinq:3-80',
 'mths_since_last_delinq:80-90',
 'mths_since_last_delinq:90-105',
 'mths_since_last_delinq:105-116',
 'mths_since_last_delinq:>=116',
 'dti:<2',
 'dti:2-4',
 'dti:4-10',
 'dti:10-15',
 'dti:15-20',
 'dti:20-25',
 'dti:25-30',
 'dti:30-35',
 'dti:35-40',
 'dti:>40',
 'mths_since_last_record:Missing',
 'mths_since_last_record:0-3',
 'mths_since_last_record:3-85',
 'mths_since_last_record:85-120',
 'mths_since_last_record:>=120']

In [10]:
# # list of features created before in preprocessing steps(removeing factor columns)

# feature_selection=[
# 'grade:A',
# 'grade:B',
# 'grade:C',
# 'grade:D',
# 'grade:E',
# 'grade:F',
# 'grade:G',
# 'home_ownership:RENT_OTHER_NONE_ANY',
# 'home_ownership:OWN',
# 'home_ownership:MORTGAGE',
# 'addr_state:ND_NE_IA_NC_FL_HI_AL',
# 'addr_state:NM_VA',
# 'addr_state:OK_TN_MO_LA_MD_NC',
# 'addr_state:UT_KY_AZ_NJ',
# 'addr_state:AR_MI_PA_OH_MN',
# 'addr_state:RI_MA_DE_SD_IN',
# 'addr_state:GA_WA_OR',
# 'addr_state:WI_MT',
# 'addr_state:IL_CT',
# 'addr_state:KS_SC_CO_VT_AK_MS',
# 'addr_state:WV_NH_WY_DC_ME_ID',
# 'verification_status:Verified',
# 'verification_status:Source Verified',
# 'verification_status:Not Verified',
#     'purpose:educ_sm_b_wedd_ren_en_mov_house',
#     'purpose:small_business',
#     'purpose:educational',
#     'purpose:renewable_energy',
#     'purpose:moving',
#     'purpose:house',
#     'purpose:other',
#     'purpose:medical',
#     'purpose:wedding',
#     'purpose:vacation',
#     'purpose:debt_consolidation',
#     'purpose:home_improvement',
#     'purpose:major_purchase',
#     'purpose:car',
#     'purpose:credit_card',
#     'initial_list_status:f',
#     'initial_list_status:w',
# 'term:36',
# 'term:60',
# 'emp_length:0',
# 'emp_length:1',
# 'emp_length:2-4',
# 'emp_length:5-6',
# 'emp_length:7-9',
# 'emp_length:10',
# 'mths_since_issued_date:<38',
# 'mths_since_issued_date:38-39',
# 'mths_since_issued_date:40-41',
# 'mths_since_issued_date:42-48',
# 'mths_since_issued_date:49-52',
# 'mths_since_issued_date:53-64',
# 'mths_since_issued_date:65-84',
# 'mths_since_issued_date:>84',
# 'int_rate:<9.548',
# 'int_rate:9.548-12.025',
# 'int_rate:12.025-15.74',
# 'int_rate:15.74-20.281',
# 'int_rate:20.281',
#     'mths_since_earliest_cr_line:<140',
#     'mths_since_earliest_cr_line:140-239',
#     'mths_since_earliest_cr_line:240-339',
#     'mths_since_earliest_cr_line:340-439',
#     'mths_since_earliest_cr_line:440-539',
#     'mths_since_earliest_cr_line:540-600',
#     'mths_since_earliest_cr_line:>600',
#     'delinq_2yrs:0',
#     'delinq_2yrs:0-3',
#     'delinq_2yrs:3-7',
#     'delinq_2yrs:7-9',
#     'delinq_2yrs:9-11',
#     'delinq_2yrs:11-12',
#     'delinq_2yrs:12-18',
#     'delinq_2yrs:>18',
#     'inq_last_6mths:0',
#     'inq_last_6mths:0-3',
#     'inq_last_6mths:3-7',
#     'inq_last_6mths:7-9',
#     'inq_last_6mths:9-11',
#     'inq_last_6mths:11-12',
#     'inq_last_6mths:12-18',
#     'inq_last_6mths:18',
#     'open_acc:=0',
#     'open_acc:0-1',
#     'open_acc:1-4',
#     'open_acc:4-28',
#     'open_acc:28-35',
#     'open_acc:35-45',
#     'open_acc:>45',
#     'pub_rec:0-3',
#     'pub_rec:3-6',
#     'pub_rec:6-8',
#     'pub_rec:8',
#     'total_acc:<0',
#     'total_acc:0-4',
#     'total_acc:4-62',
#     'total_acc:62-72',
#     'total_acc:72-85',
#     'total_acc:>85',
#     'acc_now_delinq:<0',
#     'acc_now_delinq:0-1',
#     'acc_now_delinq:1-2',
#     'acc_now_delinq:2-3',
#     'acc_now_delinq:3-5',
#     'acc_now_delinq:>5',
# 'annual_inc:<20k',
# 'annual_inc:20k-30k',
# 'annual_inc:30k-40k',
# 'annual_inc:40k-50k',
# 'annual_inc:50k-60k',
# 'annual_inc:60k-70k',
# 'annual_inc:70k-80k',
# 'annual_inc:80k-90k',
# 'annual_inc:90k-100k',
# 'annual_inc:100k-120k',
# 'annual_inc:120k-140k',
# 'annual_inc:>140k',
# 'total_rev_hi_lim:<=5k',
# 'total_rev_hi_lim:5k_10k',
# 'total_rev_hi_lim:10k_20k',
# 'total_rev_hi_lim:20k_30k',
# 'total_rev_hi_lim:30k_40k',
# 'total_rev_hi_lim:40k_50k',
# 'total_rev_hi_lim:50k_65k',
# 'total_rev_hi_lim:65k_80k',
# 'total_rev_hi_lim:>80k',
# 'dti:<5',
# 'dti:5-10',
# 'dti:10-15',
# 'dti:15-20',
# 'dti:20-25',
# 'dti:25-30',
# 'dti:30-35',
# 'dti:35-40',
# 'dti:>40',
# 'mths_since_last_delinq:Missing',
# 'mths_since_last_delinq:0-3',
# 'mths_since_last_delinq:4-30',
# 'mths_since_last_delinq:31-56',
# 'mths_since_last_delinq:>=57',
# 'mths_since_last_record:Missing',
# 'mths_since_last_record:0-90',
# 'mths_since_last_record:90-120',
# 'mths_since_last_record:120-127',
# 'mths_since_last_record:>=127','loan_amnt','funded_amnt','int_rate','installment']



In [11]:
#assign them into a inputs_train_with_ref_category with only those columns which are in the list(fetaure selection)
inputs_train_with_ref_category = load_data_inputs_train.reindex(feature_selection, axis='columns')
inputs_train_with_ref_category

,loan_amnt,funded_amnt,int_rate,installment,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,...,dti:20-25,dti:25-30,dti:30-35,dti:35-40,dti:>40,mths_since_last_record:Missing,mths_since_last_record:0-3,mths_since_last_record:3-85,mths_since_last_record:85-120,mths_since_last_record:>=120
403678,24000,24000,11.99,533.75,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
241492,30000,30000,12.39,673.27,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
15864,2400,2400,13.49,81.44,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
320277,12000,12000,13.98,410.02,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
299829,5000,5000,11.67,165.29,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73349,10000,10000,26.06,299.77,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
371403,22650,22650,18.99,830.15,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
312201,11400,11400,6.03,346.97,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
267336,8000,8000,13.35,270.91,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [12]:
#check the basic info
display(inputs_train_with_ref_category.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373028 entries, 403678 to 128037
Columns: 140 entries, loan_amnt to mths_since_last_record:>=120
dtypes: float64(2), int64(138)
memory usage: 401.3 MB


None

In [13]:
#check the inputs_train_with_ref_category
inputs_train_with_ref_category.head()

,loan_amnt,funded_amnt,int_rate,installment,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,...,dti:20-25,dti:25-30,dti:30-35,dti:35-40,dti:>40,mths_since_last_record:Missing,mths_since_last_record:0-3,mths_since_last_record:3-85,mths_since_last_record:85-120,mths_since_last_record:>=120
403678,24000,24000,11.99,533.75,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
241492,30000,30000,12.39,673.27,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
15864,2400,2400,13.49,81.44,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
320277,12000,12000,13.98,410.02,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
299829,5000,5000,11.67,165.29,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
#create a list of reference categories selecting only one column from each parent column
ref_categories= ['grade:G',
               'home_ownership:RENT_OTHER_NONE_ANY',
               'addr_state_NE',
               'verification_status:Verified',
                'purpose:educ_sm_b_ren_en_mov_house_other_med',
                'initial_list_status:f',
                'term:60',
                'emp_length:0',
                'mths_since_issued_date:<116',
                'int_rate:20.281',
                'mths_since_earliest_cr_line:<140',
                'delinq_2yrs:>18',
                'inq_last_6mths:0',
                'open_acc:=0',
                'pub_rec:0-6',
                'total_acc:>99.84',
                'acc_now_delinq:>3',
                'total_rev_hi_lim:<6.4k',
                'annual_inc:<8k',
                'dti:<2',
                'mths_since_last_delinq:Missing',
                'mths_since_last_record:Missing',
               ]

In [15]:
#drop all the reference column from inputs_train_with_ref_category
inputs_train= inputs_train_with_ref_category.drop(ref_categories, axis = 1)
inputs_train.head()

,loan_amnt,funded_amnt,int_rate,installment,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,...,dti:15-20,dti:20-25,dti:25-30,dti:30-35,dti:35-40,dti:>40,mths_since_last_record:0-3,mths_since_last_record:3-85,mths_since_last_record:85-120,mths_since_last_record:>=120
403678,24000,24000,11.99,533.75,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
241492,30000,30000,12.39,673.27,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
15864,2400,2400,13.49,81.44,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
320277,12000,12000,13.98,410.02,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
299829,5000,5000,11.67,165.29,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


similar feature selection process for test input data

In [16]:
#preparing the test data with only those variables which are in feature selection
inputs_test_with_ref_category= load_data_inputs_test.reindex(feature_selection, axis='columns')

In [17]:
#dropping the reference categories in the inputs_test_with_ref_category
inputs_test= inputs_test_with_ref_category.drop(ref_categories,axis=1)

#check the inputs_test dataframe
inputs_test.head()

,loan_amnt,funded_amnt,int_rate,installment,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,...,dti:15-20,dti:20-25,dti:25-30,dti:30-35,dti:35-40,dti:>40,mths_since_last_record:0-3,mths_since_last_record:3-85,mths_since_last_record:85-120,mths_since_last_record:>=120
421662,16000,16000,12.49,535.19,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
345847,24000,24000,16.29,587.34,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
87245,19125,19125,23.10,540.25,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
422411,15000,15000,13.65,510.12,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
131614,24000,24000,23.76,687.10,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [18]:
#to display max columns
pd.options.display.max_rows = None

In [19]:
#checking for null values in training data
inputs_train.isna().sum()

loan_amnt                                                                                0
funded_amnt                                                                              0
int_rate                                                                                 0
installment                                                                              0
grade:A                                                                                  0
grade:B                                                                                  0
grade:C                                                                                  0
grade:D                                                                                  0
grade:E                                                                                  0
grade:F                                                                                  0
home_ownership:MORTGAGE                                                                  0

In [20]:
inputs_test.isna().sum()

loan_amnt                                                                                0
funded_amnt                                                                              0
int_rate                                                                                 0
installment                                                                              0
grade:A                                                                                  0
grade:B                                                                                  0
grade:C                                                                                  0
grade:D                                                                                  0
grade:E                                                                                  0
grade:F                                                                                  0
home_ownership:MORTGAGE                                                                  0

PD MODLE ESTIMATION

Logistic Regression

In [21]:
#create an object of logistic regression with max_iter=200
model_lr = LogisticRegression(max_iter= 200)

In [22]:
#convert the inputs_train into float
inputs_train= inputs_train.astype(float)

In [23]:
#convert the load_data_targets_train into float
load_data_targets_train= load_data_targets_train.astype(float)

In [24]:
#fit the LR object onto the inputs_train,load_data_targets_train
model_lr.fit(inputs_train,load_data_targets_train)

LogisticRegression(max_iter=200)

In [25]:
#predict the inputs_test
y_hat_test = model_lr.predict(inputs_test)
np.unique(y_hat_test)

array([1.])

In [26]:
#astype the y_hat_test into int
a= y_hat_test.astype(int)

In [27]:
#predict the predict_proba of inputs_test
y_hat_test_proba = model_lr.predict_proba(inputs_test)

In [28]:
#print the y_hat_test_proba
y_hat_test_proba

array([[0.11807338, 0.88192662],
       [0.0810954 , 0.9189046 ],
       [0.05249038, 0.94750962],
       ...,
       [0.18656939, 0.81343061],
       [0.03322989, 0.96677011],
       [0.18573928, 0.81426072]])

In [29]:
#take only one column from the y_hat_test_proba
y_hat_test_proba[: ,1]

array([0.88192662, 0.9189046 , 0.94750962, ..., 0.81343061, 0.96677011,
       0.81426072])

In [30]:
#take only one column[- probability for 1] from the y_hat_test_proba
y_hat_test_proba= y_hat_test_proba[: ,1]

In [31]:
#assign load_data_targets_test into a new variable
loan_data_targets_test_temp = load_data_targets_test
loan_data_targets_test_temp.head(5)

,good_bad
421662,1
345847,1
87245,1
422411,1
131614,1


In [32]:
#reset the index of load_data_targets_test_temp
loan_data_targets_test_temp.reset_index(drop= True, inplace= True)
loan_data_targets_test_temp.head()

,good_bad
0,1
1,1
2,1
3,1
4,1


In [33]:
#concat the original output and predicted probability in one dataframe
df_actual_predicted_probs = pd.concat([loan_data_targets_test_temp,pd.DataFrame(y_hat_test_proba)], axis=1)

In [34]:
#change the column names into meaningful names
df_actual_predicted_probs.columns = ['load_data_targets_test','y_hat_test_proba']

In [35]:
#check the shape of df_actual_predicted_probs
df_actual_predicted_probs.shape

(93257, 2)

In [36]:
#match both the index
df_actual_predicted_probs.index=load_data_inputs_test.index

In [37]:
#print the head of df_actual_predicted_probs
df_actual_predicted_probs.head()

,load_data_targets_test,y_hat_test_proba
421662,1,0.881927
345847,1,0.918905
87245,1,0.947510
422411,1,0.887450
131614,1,0.960882


In [38]:
#take the threshold as 0.9 as assign 0 and 1 accordingly using the np.where function
tr = 0.8
df_actual_predicted_probs['y_hat_test']= np.where(df_actual_predicted_probs['y_hat_test_proba'] > tr, 1,0)

In [39]:
#print the crosstab of actual and prdicted values
pd.crosstab(df_actual_predicted_probs['load_data_targets_test'],df_actual_predicted_probs['y_hat_test'],
    rownames = ['Actual'], colnames = ['Predicted'])

Predicted,0,1
Actual,,
0,870,9408
1,13671,69308


# Generalized function for model building and checking performance

In [40]:
def x_tr_model(model,x_train,y_train,x_test,y_test,tr):
    model.fit(x_train,y_train)
    pred_y = model.predict(x_test)
    y_hat_test_proba = model.predict_proba(inputs_test)
    y_hat_test_proba=y_hat_test_proba[: ,1]
    loan_data_targets_test_temp = y_test
    loan_data_targets_test_temp.reset_index(drop= True, inplace= True)
    df_actual_predicted_probs = pd.concat([loan_data_targets_test_temp,pd.DataFrame(y_hat_test_proba)], axis=1)
    df_actual_predicted_probs.columns =  ['load_data_targets_test','y_hat_test_proba']
    df_actual_predicted_probs.index=load_data_inputs_test.index
    df_actual_predicted_probs['y_hat_test']= np.where(df_actual_predicted_probs['y_hat_test_proba'] > tr, 1,0)
    print(f'unique Probability values are:{np.unique(y_hat_test_proba)}')

    return pd.crosstab(df_actual_predicted_probs['load_data_targets_test'],df_actual_predicted_probs['y_hat_test'],rownames = ['Actual'], colnames = ['Predicted'])



In [41]:
#confusion matrix result for Logistic Regression model with 150 max_iter
x_tr_model(LogisticRegression(max_iter=150),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

unique Probability values are:[0.53200883 0.53648241 0.55467411 ... 0.98938389 0.98943534 0.98954576]


Predicted,0,1
Actual,,
0,2722,7556
1,33570,49409


In [42]:
#confusion matrix result with RandomForest
x_tr_model(RandomForestClassifier(),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

unique Probability values are:[0.32       0.33       0.36       0.37       0.39       0.4
 0.41       0.44       0.45       0.46       0.47       0.48
 0.49       0.5        0.51       0.52       0.53       0.54
 0.55       0.56       0.57       0.58       0.59       0.6
 0.6025     0.61       0.62       0.62666667 0.63       0.64
 0.65       0.66       0.67       0.68       0.69       0.69666667
 0.7        0.71       0.716      0.72       0.72333333 0.73
 0.74       0.745      0.74833333 0.75       0.76       0.7625
 0.765      0.77       0.779      0.78       0.785      0.79
 0.79166667 0.79666667 0.8        0.80516667 0.81       0.8175
 0.81833333 0.82       0.82333333 0.83       0.83333333 0.83833333
 0.84       0.84083333 0.845      0.846      0.85       0.85666667
 0.8575     0.86       0.86666667 0.87       0.87333333 0.875
 0.876      0.8775     0.88       0.8825     0.885      0.89
 0.89666667 0.899      0.9        0.905      0.91       0.91333333
 0.9175     0.92       0.925

Predicted,0,1
Actual,,
0,5230,5048
1,23269,59710


In [43]:
#confusion matrix result with DT
x_tr_model(DecisionTreeClassifier(),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

unique Probability values are:[0.  0.5 1. ]


Predicted,0,1
Actual,,
0,1849,8429
1,10159,72820


In [44]:
#confusion matrix result with GBC
x_tr_model(GradientBoostingClassifier(),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

unique Probability values are:[0.34111118 0.37369914 0.40034705 ... 0.98190294 0.9825717  0.98277502]


Predicted,0,1
Actual,,
0,4525,5753
1,16434,66545


### **Hyper-Parameter Tuning**

In [45]:
# Helper function to perform hyper parameter tunning with RandomizedSearchCV
def random_Search(model,X_train, Y_train,param_grid):


    # Random search of parameters, using 3 fold cross validation,
    # search across 60 different combinations, and use all available cores
    random = RandomizedSearchCV(estimator = model, param_distributions = param_grid, n_iter = 10, cv = 2, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
    random.fit(X_train, Y_train)
    print(random.best_params_)


In [46]:
# for decision tree classifier
# create parameters dict for tunning
DTR_para_grid = {
                  "splitter":["best","random"],
            "max_depth" : [1,3,12],
           "min_samples_leaf":[1,4,5,9],
           "min_weight_fraction_leaf":[0.1,0.3,0,0.7,0.9],
           "max_features":["auto","log2"],
           "max_leaf_nodes":[10,40,60,80]
              }

# passing data for hyper parameter tunning with Randomized search cv
random_Search(DecisionTreeClassifier(),inputs_train,load_data_targets_train, DTR_para_grid)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
{'splitter': 'best', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 9, 'max_leaf_nodes': 10, 'max_features': 'auto', 'max_depth': 3}


In [ ]:
# for random forest classifier
# create parameters dict for tunning
rf_para_grid = {'n_estimators': [int(x) for x in np.arange(200,500,100)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 80, num = 11)],
               'min_samples_leaf': [1, 2, 4]

               }

# passing data for hyper parameter tunning with Randomized search cv
random_Search(RandomForestClassifier(),inputs_train,load_data_targets_train, rf_para_grid)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


In [ ]:
#creating a DT model with best parameters
x_tr_model(DecisionTreeClassifier(splitter= 'best', min_weight_fraction_leaf= 0.1, min_samples_leaf= 9, max_leaf_nodes= 10, max_features='auto', max_depth= 3),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

In [ ]:
#creating a Random Forest model with best parameters
x_tr_model(RandomForestClassifier(),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

In [ ]:
# for Gradient Boosting classifier
# create parameters dict for tunning
gb_parameters = {
    "n_estimators":[50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10]
}

# passing data for hyper parameter tunning with Randomized search cv
random_Search(GradientBoostingClassifier(),inputs_train,load_data_targets_train, gb_parameters)

In [ ]:
#creating a Gradient Boosting model with best parameters
x_tr_model(GradientBoostingClassifier(),inputs_train,load_data_targets_train,inputs_test,load_data_targets_test,0.85)

In [ ]:
#create a model for saving the model in further operation
grad_boss = GradientBoostingClassifier(n_estimators= 50, max_depth= 1, learning_rate= 0.01)
grad_boss.fit(inputs_train,load_data_targets_train)

In [ ]:
#save the gradient boosting model for further use
#pickle.dump(grad_boss,open('pd_model_gradient.sav','wb'))